In [2]:
%load_ext autoreload
%autoreload 2
import os
import sys
module_path = os.path.abspath(os.path.join('./xclip'))
sys.path.insert(0, module_path)

In [187]:
import xclip
import xclip.models.xclip as xclip
from xclip.utils.logger import create_logger
logger = create_logger('./')
import numpy as np
import torch
from xclip.utils.config import  get_config  
from xclip.models.cct import load
import xclip.datasets.build as build
import cv2
import timm
import open_clip
from xclip.clip.clip import load_vit
import xclip.utils.tools as tools
import time
from xclip.models.repnet import RepNet

In [188]:
class Args:
    def __init__(self):
        self.config = './xclip/configs/countix/countix.yaml'
        self.opts = None
        self.auto_resume = False
        self.batch_size=4
        self.pretrained=None
        self.resume=False
        self.accumulation_steps=None
        self.output=None
        self.only_test=False
        self.local_rank=None
args = Args()
config=get_config(args)


=> merge config from ./xclip/configs/countix/countix.yaml


In [5]:
embed_model = load_vit(None,'ViT-B-16', device="cpu", jit=False,pretrained='laion400m_e31')

[2024-12-01 07:04:33 root](factory.py 296): INFO Loaded ViT-B-16 model config.
[2024-12-01 07:04:35 urllib3.connectionpool](connectionpool.py 1021): DEBUG Starting new HTTPS connection (1): huggingface.co:443
[2024-12-01 07:04:35 urllib3.connectionpool](connectionpool.py 474): DEBUG https://huggingface.co:443 "HEAD /timm/vit_base_patch16_clip_224.laion400m_e31/resolve/main/open_clip_model.safetensors HTTP/1.1" 302 0
[2024-12-01 07:04:35 root](factory.py 383): INFO Loading pretrained ViT-B-16 weights (laion400m_e31).


In [189]:
train_data,train_loader = build.build_counitxdataloader(config)


In [7]:
STRIDES = [1,2,3]


In [69]:
model = RepNet()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001,eps=1e-8)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)  
critertion = torch.nn.CrossEntropyLoss()
start_epoch, max_accuracy = 0, 0.0

In [190]:
model.train()
for idx, batch_data in enumerate(train_loader):
        images = batch_data["imgs"]
        labels = batch_data["label"]
        count   = batch_data["count"]
        count_out = [0] * images.shape[0]
        period_length = [0] * images.shape[0]
        confidence = [0] * images.shape[0]
        period_count = [0] * images.shape[0]
        periodicity_score = [0]  * images.shape[0]
        best_confidence = [0] * images.shape[0]
        inputs = images.view(-1,images.shape[2],images.shape[3],images.shape[4])
        with torch.no_grad():
                outputs = embed_model(inputs)
         


objc[68046]: Class AVFFrameReceiver is implemented in both /Users/kanantharaman/anaconda3/lib/python3.11/site-packages/av/.dylibs/libavdevice.60.3.100.dylib (0x1496900f8) and /Users/kanantharaman/anaconda3/lib/python3.11/site-packages/decord/.dylibs/libavdevice.58.5.100.dylib (0x1613ef010). One of the two will be used. Which one is undefined.
objc[68046]: Class AVFAudioReceiver is implemented in both /Users/kanantharaman/anaconda3/lib/python3.11/site-packages/av/.dylibs/libavdevice.60.3.100.dylib (0x149690148) and /Users/kanantharaman/anaconda3/lib/python3.11/site-packages/decord/.dylibs/libavdevice.58.5.100.dylib (0x1613ef060). One of the two will be used. Which one is undefined.


12/01 22:01:17 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
12/01 22:01:17 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.


In [57]:
count   = batch_data["count"]
count_out = [0] * images.shape[0]
period_length = [0] * images.shape[0]
confidence = [0] * images.shape[0]
period_count = [0] * images.shape[0]
periodicity_score = [0]  * images.shape[0]
best_confidence = [0] * images.shape[0]

In [203]:
outputs1=outputs[::2]

In [154]:
outputs1 = outputs1.view(images.shape[0],-1,outputs.shape[1])
#outputs1=outputs1.split(64,dim=1)

In [60]:

raw_period_length, raw_periodicity_score, embeddings = [], [], []
for i in range(len(outputs1)):  # Process each batch separately to avoid OOM
                    batch_period_length, batch_periodicity, batch_embeddings = model(outputs1[i])
                    raw_period_length.append(batch_period_length.cpu())
                    raw_periodicity_score.append(batch_periodicity.cpu())
                    embeddings.append(batch_embeddings.cpu()) 
            # Post-process results
raw_period_length, raw_periodicity_score, embeddings = torch.cat(raw_period_length,1), torch.cat(raw_periodicity_score,1), torch.cat(embeddings,1)
for i in range(raw_period_length.shape[0]):
                confidence[i], period_length[i], period_count[i], periodicity_score[i] = model.get_counts(raw_period_length[i], raw_periodicity_score[i], 1)
                if best_confidence[i] is None or confidence[i] > best_confidence[i]:
                    count_out[i] = [period_count[i][-1].item()]
                    best_confidence[i] = confidence[i]
print(count_out,count)

[[0.0], [0.0], [0.0], [0.0]] tensor([[2],
        [4],
        [4],
        [3]])


In [ ]:
def plot_heatmap(dist: np.ndarray, log_scale: bool = False) -> np.ndarray:
    """Plot the temporal self-similarity matrix into an OpenCV image."""
    np.fill_diagonal(dist, np.nan)
    if log_scale:
        dist = np.log(1 + dist)
    dist = -dist # Invert the distance
    zmin, zmax = np.nanmin(dist), np.nanmax(dist)
    heatmap = (dist - zmin) / (zmax - zmin) # Normalize into [0, 1]
    heatmap = np.nan_to_num(heatmap, nan=1)
    heatmap = np.clip(heatmap * 255, 0, 255).astype(np.uint8)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_VIRIDIS)
    return heatmap

In [155]:
dist  = torch.cdist(outputs1,outputs1)**2

In [94]:
import matplotlib.pyplot as plt


[2024-12-01 20:53:55 matplotlib](__init__.py 305): DEBUG matplotlib data path: /Users/kanantharaman/anaconda3/lib/python3.11/site-packages/matplotlib/mpl-data
[2024-12-01 20:53:55 matplotlib](__init__.py 305): DEBUG CONFIGDIR=/Users/kanantharaman/.matplotlib
[2024-12-01 20:53:55 matplotlib](__init__.py 1479): DEBUG interactive is False
[2024-12-01 20:53:55 matplotlib](__init__.py 1480): DEBUG platform is darwin
[2024-12-01 20:53:55 matplotlib](__init__.py 305): DEBUG CACHEDIR=/Users/kanantharaman/.matplotlib
[2024-12-01 20:53:55 matplotlib.font_manager](font_manager.py 1543): DEBUG Using fontManager instance from /Users/kanantharaman/.matplotlib/fontlist-v330.json


In [ ]:
dist[0][:64,:64].shape

torch.Size([200, 200])

In [186]:
cv2.imwrite(f'heatmap_{0}_{0}.png', plot_heatmap(dist[0][:55,:55].numpy()))

True

In [198]:
outputs1=outputs[::3]
outputs1 = outputs1.view(images.shape[0],-1,outputs.shape[1])
dist  = torch.cdist(outputs1,outputs1)**2
for i in range(4):
        cv2.imwrite(f'heatmap_{i}_{i}.png', plot_heatmap(dist[i].numpy()))


In [205]:
def train_one_epoch(epoch, model, criterion, optimizer, lr_scheduler, train_loader, text_labels, config, logger):
    model.train()
    total_loss = 0.0
    optimizer.zero_grad()
    
    num_steps = len(train_loader)
    batch_time = tools.AverageMeter()
    tot_loss_meter = tools.AverageMeter()
    
    start = time.time()
    end = time.time()
    
    for idx, batch_data in enumerate(train_loader):
        images = batch_data["imgs"]
        labels = batch_data["label"]
        count   = batch_data["count"]
        count_out = [0] * images.shape[0]
        period_length = [0] * images.shape[0]
        confidence = [0] * images.shape[0]
        period_count = [0] * images.shape[0]
        periodicity_score = [0]  * images.shape[0]
        best_confidence = [0] * images.shape[0]
        inputs = images.view(-1,images.shape[2],images.shape[3],images.shape[4])
        with torch.no_grad():
                embeds = embed_model(inputs)
        print(inputs.shape)
        for stride in STRIDES:
            stride_frames = embeds[::stride]
            outputs = stride_frames.view(images.shape[0],-1,stride_frames.shape[1])
            outputs=outputs.split(64,dim=1)
            raw_period_length, raw_periodicity_score, embeddings = [], [], []
            for i in range(len(outputs)):  # Process each batch separately to avoid OOM
                    batch_period_length, batch_periodicity, batch_embeddings = model(outputs[i])
                    raw_period_length.append(batch_period_length.cpu())
                    raw_periodicity_score.append(batch_periodicity.cpu())
                    embeddings.append(batch_embeddings.cpu()) 
            # Post-process results
            raw_period_length, raw_periodicity_score, embeddings = torch.cat(raw_period_length,1), torch.cat(raw_periodicity_score,1), torch.cat(embeddings,1)
            for i in range(raw_period_length.shape[0]):
                confidence[i], period_length[i], period_count[i], periodicity_score[i] = model.get_counts(raw_period_length[i], raw_periodicity_score[i], stride)
                if best_confidence[i] is None or confidence[i] > best_confidence[i]:
                    count_out[i] = [period_count[i][-1].item()]
                    best_confidence[i] = confidence[i]
    print(count_out,count)
    loss = criterion(torch.tensor(count_out,requires_grad=True), count.float())
    print(loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    lr_scheduler.step()
    total_loss += loss.item()
    return total_loss


In [206]:
start_epoch, max_accuracy = 0, 0.0
for epoch in range(start_epoch, 2):
    train_one_epoch(1, model, critertion, optimizer, scheduler, train_loader, None, config, logger)

objc[74584]: Class AVFFrameReceiver is implemented in both /Users/kanantharaman/anaconda3/lib/python3.11/site-packages/av/.dylibs/libavdevice.60.3.100.dylib (0x14b4220f8) and /Users/kanantharaman/anaconda3/lib/python3.11/site-packages/decord/.dylibs/libavdevice.58.5.100.dylib (0x15008f010). One of the two will be used. Which one is undefined.
objc[74584]: Class AVFAudioReceiver is implemented in both /Users/kanantharaman/anaconda3/lib/python3.11/site-packages/av/.dylibs/libavdevice.60.3.100.dylib (0x14b422148) and /Users/kanantharaman/anaconda3/lib/python3.11/site-packages/decord/.dylibs/libavdevice.58.5.100.dylib (0x15008f060). One of the two will be used. Which one is undefined.


12/01 22:24:32 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
12/01 22:24:32 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.
torch.Size([1536, 3, 224, 224])
[[0.0], [0.0], [0.0], [0.0]] tensor([[2],
        [4],
        [4],
        [3]])
tensor(-0., grad_fn=<DivBackward1>)


objc[75485]: Class AVFFrameReceiver is implemented in both /Users/kanantharaman/anaconda3/lib/python3.11/site-packages/av/.dylibs/libavdevice.60.3.100.dylib (0x1408060f8) and /Users/kanantharaman/anaconda3/lib/python3.11/site-packages/decord/.dylibs/libavdevice.58.5.100.dylib (0x145473010). One of the two will be used. Which one is undefined.
objc[75485]: Class AVFAudioReceiver is implemented in both /Users/kanantharaman/anaconda3/lib/python3.11/site-packages/av/.dylibs/libavdevice.60.3.100.dylib (0x140806148) and /Users/kanantharaman/anaconda3/lib/python3.11/site-packages/decord/.dylibs/libavdevice.58.5.100.dylib (0x145473060). One of the two will be used. Which one is undefined.


12/01 22:27:47 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
12/01 22:27:47 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.
torch.Size([1536, 3, 224, 224])
[[0.0], [0.0], [0.0], [0.0]] tensor([[2],
        [4],
        [4],
        [3]])
tensor(-0., grad_fn=<DivBackward1>)
